In [ ]:
import os
import sys

import polars as pl


sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import StashAppClient, get_stashapp_client


stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
stash_tags = stash_client.get_tags_by_names([
    "Full Movie",
    "Compilation",
    "Multiple Sex Scenes in a Scene",
    "Behind the Scenes",
    "TV Series",
    "Non-Sex Performer",
    "Virtual Sex",
    "Missing Performer (Male)",
    "Missing Performer (Female)",
    "Group Makeup Verified",
])

excluded_tags = [
    stash_tags.full_movie["id"],
    stash_tags.compilation["id"],
    stash_tags.multiple_sex_scenes_in_a_scene["id"],
    stash_tags.behind_the_scenes["id"],
    stash_tags.tv_series["id"],
    stash_tags.non_sex_performer["id"],
    stash_tags.virtual_sex["id"],
    stash_tags.missing_performer_male["id"],
    stash_tags.missing_performer_female["id"],
]

group_makeup_calculated_parent_tag = stash.find_tag(
    "Group Makeup Calculated", fragment="id name children { id name }"
)
group_makeup_calculated_tag_ids = [
    tag["id"] for tag in group_makeup_calculated_parent_tag["children"]
]

In [ ]:
def get_gender_makeup(performers):
    # Initialize counters for each gender
    counts = {
        "TRANSGENDER_FEMALE": 0,
        "TRANSGENDER_MALE": 0,
        "NON_BINARY": 0,
        "FEMALE": 0,
        "MALE": 0,
    }

    # Count each gender
    for performer in performers:
        counts[performer["gender"]] += 1

    # Create abbreviations mapping
    abbrev = {
        "TRANSGENDER_FEMALE": "TF",
        "TRANSGENDER_MALE": "TM",
        "NON_BINARY": "NB",
        "FEMALE": "F",
        "MALE": "M",
    }

    # Build the string
    result = ""
    for gender in [
        "TRANSGENDER_FEMALE",
        "TRANSGENDER_MALE",
        "NON_BINARY",
        "FEMALE",
        "MALE",
    ]:
        if counts[gender] > 0:
            result += f"{counts[gender]}{abbrev[gender]}"

    return result


filtered_scenes = pl.DataFrame(
    stash.find_scenes(
        {"tags": {"value": [], "modifier": "INCLUDES", "excludes": excluded_tags}},
        fragment="id title date performers { id name gender } tags { id name }",
    )
).with_columns(
    pl.col("performers")
    .list.eval(
        pl.element().filter(pl.element().struct.field("gender").is_not_null())
    )
    .alias("performers")
).filter(
    pl.col("performers").list.len() > 0
)

unique_genders = (
    filtered_scenes.explode("performers")
    .select(pl.col("performers").struct.field("gender"))
    .unique()
)
print("Unique genders found after filtering:")
print(unique_genders)

filtered_scenes

In [ ]:
# Now add the gender makeup as a new column
all_scenes_with_makeup = filtered_scenes.with_columns(
    pl.col("performers")
    .map_elements(get_gender_makeup, return_dtype=pl.Utf8)
    .alias("gender_makeup")
).with_columns(
    ("Group Makeup Calculated: " + pl.col("gender_makeup")).alias("full_tag_name")
)

all_scenes_with_makeup

In [ ]:
group_makeup_calculated_tag_names = (
    all_scenes_with_makeup.select("full_tag_name")
    .unique()
    .sort(by="full_tag_name")
)
group_makeup_calculated_tag_names

In [ ]:
all_tags = pl.DataFrame(stash.find_tags(fragment="id name"))
unique_group_makeup_calculated_tags = all_tags.join(
    group_makeup_calculated_tag_names, left_on="name", right_on="full_tag_name", how="right"
)
group_makeup_calculated_lookup = {
    row["full_tag_name"]: row["id"]
    for row in unique_group_makeup_calculated_tags.iter_rows(named=True)
}

group_makeup_calculated_parent_tag_id = stash.find_tag("Group Makeup Calculated")["id"]
group_makeup_calculated_parent_tag_id

for gender_makeup in group_makeup_calculated_tag_names.iter_rows(named=True):
    if (
        gender_makeup["full_tag_name"] in group_makeup_calculated_lookup
        and group_makeup_calculated_lookup[gender_makeup["full_tag_name"]] is not None
    ):
        continue

    tag = stash.create_tag(
        {
            "name": gender_makeup["full_tag_name"],
            "parent_ids": [group_makeup_calculated_parent_tag_id],
        }
    )
    print(f"Created tag: {tag['id']} - {tag['name']}")

group_makeup_calculated_tags_df: pl.DataFrame = pl.DataFrame(
    stash.find_tags(
        {
            "parents": {
                "value": [group_makeup_calculated_parent_tag_id],
                "modifier": "INCLUDES",
                "depth": 1,
            }
        },
        fragment="id name",
    )
)
group_makeup_calculated_tags_df

In [ ]:
# Group scenes by group_makeup_calculated_tag_id and bulk update
scene_tag_mappings = all_scenes_with_makeup.join(
    group_makeup_calculated_tags_df,
    left_on="full_tag_name",
    right_on="name",
    how="left"
).select(
    pl.col("id_right").alias("group_makeup_calculated_tag_id"),
    pl.col("id").alias("scene_id")
)

# Group by tag ID and collect scene IDs for each group
grouped_scene_updates = scene_tag_mappings.group_by("group_makeup_calculated_tag_id").agg(
    pl.col("scene_id").alias("scene_ids")
)

# Apply bulk updates for each group
for group in grouped_scene_updates.iter_rows(named=True):
    tag_id = group["group_makeup_calculated_tag_id"]
    scene_ids = group["scene_ids"]

    print(f"Adding tag {tag_id} to {len(scene_ids)} scenes")

    # Use bulk update to add the tag to all scenes in this group
    stash_client.bulk_scene_update(scene_ids, [tag_id], "ADD")

print("✅ All group makeup calculated tags have been applied to scenes!")

grouped_scene_updates
